#기본 세팅

In [1]:
pip install xlrd

In [2]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 14.9 MB/s eta 0:00:00


In [3]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 34.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import requests
import re
import konlpy
import numpy as np
from konlpy.tag import Okt
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

#데이터 불러오기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


원본데이터

In [7]:
df_original = pd.read_csv("/content/drive/MyDrive/졸업 프로젝트 - 도서추천/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/추천서비스_최종데이터(10.26).csv",
            encoding = 'utf-8-sig',
            dtype = {'ISBN' : "str"})
df = df_original.copy()

df.loc[df['요약'] == '#NAME?', '요약'] = ' '

In [8]:
#df 표시할 때 string 데이터 줄이지 말고 다 쓰기
pd.set_option('display.max_colwidth', None)
df_original[['description_naver']].iloc[3516]
df_original[['요약']].iloc[3516]


,3516
요약,"확실한 것이 없는 시대에도, 마츠모토 타이요가의 만화인 삶, 애정, 철학은 계속된다. 대형 출판사의 편집자이자 만화가들의 만화가인 시오자와는 30년 넘게 일한 출판사를 떠나 자신의 이상을 담은 잡지를 만들기로 결심한다. 이를 위해 동경했던 만화가들을 찾아가며 다시금 만화의 세계로 이끈다. 그들은 불확실한 시대 속에서 새로운 불씨를 틔워내며 치열하게 나아간다."


#CID group 하기

In [12]:
cid_data = pd.read_excel("/content/drive/MyDrive/졸업 프로젝트 - 도서추천/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/aladin_Category_CID_20210927.xls")

#열 재정의, 필요없는 행 삭제
cid_data.columns = cid_data.iloc[1]
cid_data = cid_data.drop(index = [0,1])
cid_data.head(3)

1,CID,카테고리명,몰,1Depth,2Depth,3Depth,4Depth,5Depth
2,1230,가정/요리/뷰티,국내도서,가정/요리/뷰티,NaN,NaN,NaN,NaN
3,90452,가계부,국내도서,가정/요리/뷰티,가계부,NaN,NaN,NaN
4,53471,건강요리,국내도서,가정/요리/뷰티,건강요리,NaN,NaN,NaN


In [13]:
grouped_cid = cid_data.groupby(['1Depth']).agg({'CID': lambda x: list(x)}).reset_index()

print("[1Depth] groupby 전체 length : ", len(grouped_cid))
print("CID가 하나만 속하는 데이터 :", len(grouped_cid[grouped_cid['CID'].apply(len) == 1]))
grouped_cid[50:60]

[1Depth] groupby 전체 length :  111
CID가 하나만 속하는 데이터 : 11


1,1Depth,CID
50,사전/기타,"[38419, 39068, 41818, 41819, 39200, 42527, 42569, 39497, 43664, 43663, 56750, 58429, 58430, 74374, 74473, 74474, 77523, 77530, 77529, 89066, 89069, 89068, 112488, 112489, 112490, 121481, 121482, 121483, 129772, 129774, 129775, 138305, 138306, 138307, 148678, 148679, 148680, 167693, 160510, 160511, 160512, 174957, 173175, 173176, 173177, 188423, 186632, 186001, 44524, 44525, 44526, 44527, 44528, 44529, 44530, 44531, 44532, 44533, 44534, 44535, 39186, 42495, 42481, 42484, 42486, 42487, 42482, 42483, 42485, 38840, 40412, 41027, 41024, 45836, 44867, 44868, 44869, 44870, 41025, 41288, 41026, 38842, 38856, 41067, 41061, 41065, 41062, 41064, 41063, 41066, 38767, 40775, 40773, 40772, 40774, 39082, 38841, 41029, 41031, 41028, ...]"
51,사회과학,"[798, 51052, 50999, 51121, 51117, 51111, 69510, 51113, 51119, 51050, 51396, 51411, 51401, 66543, 51407, 51008, 51046, 51364, 51372, 51360, 51356, 51348, 51359, 51350, 51366, 51368, 51369, 51362, 51352, 51354, 50992, 51068, 51061, 51066, 51064, 51059, 50995, 51086, 51078, 51088, 51084, 51073, 51090, 51075, 51074, 51081, 51004, 51141, 51161, 51162, 51159, 51150, 51145, 51147, 51154, 51149, 51144, 51152, 51001, 51129, 51136, 65238, 51131, 51134, 51026, 51314, 51308, 51311, 51304, 51306, 51041, 51346, 51344, 51345, 51347, 51037, 51341, 51343, 51326, 51330, 51333, 51336, 51338, 51016, 51231, 51240, 51241, 51238, 51251, 51248, 51246, 51243, 51224, 51165, 51166, 51217, 51228, 51220, 51233, 51226, ...]"
52,상품권,[28640]
53,성인,[6548]
54,소설/시/희곡,"[1, 50930, 89481, 89482, 50922, 50927, 51116, 88136, 106545, 54719, 51080, 162391, 51107, 121541, 89171, 112097, 51083, 114691, 111196, 110488, 51071, 51077, 87919, 128346, 112098, 51099, 51102, 51070, 138767, 51105, 148769, 148770, 109725, 89172, 89168, 154903, 89166, 51114, 168549, 51106, 130122, 115694, 55624, 51069, 89167, 89169, 89170, 52650, 50935, 181723, 181727, 51126, 51125, 50932, 50951, 51234, 51218, 51215, 52653, 51225, 51230, 51221, 51229, 157672, 51223, 51227, 51213, 51232, 52654, 50945, 51193, 51179, 51206, 51191, 51204, 51319, 51320, 51321, 51322, 51323, 51325, 51328, 51332, 51186, 51190, 51180, 51196, 51184, 51182, 51201, 51309, 51305, 51302, 51303, 51300, 51297, 51295, 51316, 51313, 51198, ...]"
55,수험서,"[90843, 97459, 97460, 97461, 97462, 97463, 97464, 97465, 97466, 97467, 97468, 97469, 97470, 97471, 97472, 97473, 97474, 97475, 97476, 97477, 136305, 97478, 97479, 97480, 136306, 97481, 97482, 97483, 97484, 97485, 97486, 97487, 139466, 97488]"
56,수험서/자격증,"[1383, 34582, 34612, 34613, 34625, 34626, 34628, 34629, 34630, 34631, 34632, 34633, 34634, 34635, 34636, 34637, 34610, 34611, 37033, 34617, 34618, 34619, 34620, 34621, 34622, 34623, 34624, 34638, 34639, 35750, 36957, 34925, 34595, 34596, 34597, 34598, 34599, 34600, 34601, 34602, 34603, 34604, 36167, 34609, 48308, 140176, 75373, 75374, 75556, 75557, 34583, 34584, 34585, 34586, 34587, 34588, 34589, 34590, 34592, 75555, 75372, 80108, 80228, 80109, 140196, 140197, 123343, 140194, 34608, 140200, 140201, 140202, 140203, 140204, 140205, 140206, 140207, 140208, 140199, 52055, 34591, 36960, 34726, 34727, 34728, 34729, 34730, 35943, 36822, 34697, 34605, 140198, 123344, 123342, 34640, 36286, 34927, 140209, 140210, 140211, ...]"
57,스페인 도서,"[28254, 159297, 75161, 75167, 75174, 75172, 75169, 159292, 75157, 75155, 75162, 75153, 75173, 75164, 75158, 107528, 107529, 75159, 159299, 75163, 75170, 75171, 75160, 75166, 75165, 75154, 75168, 75156, 75175]"
58,알라딘오디오북,"[158585, 167553, 175974, 167547, 176194, 167549, 167542, 167546, 176195, 167545, 167544, 167543, 167552, 167550, 176193, 168610, 167548, 176191, 167551, 176192, 176076, 167554, 182332, 167555]"
59,애니메이션,"[6550, 69740, 55652, 56376, 89266, 66606, 69739, 54001, 70578]"


In [14]:
#빠른 매핑을 위한 딕셔너리 만들기
depth2_dict = {}
for i, items in enumerate(grouped_cid['CID']):
  for id in items:
    depth2_dict[id] = grouped_cid['1Depth'].index[i]

#적용
df['genre_code'] = df['genre_id'].map(depth2_dict).astype('Int32')
df['genre_code'] = df['genre_code'].fillna(0).astype('Int32')
df = df[['ISBN','genre_code', 'lda_result', '요약']]
df.head(3)

,ISBN,genre_code,lda_result,요약
0,9788976263728,40,38,"해당 텍스트는 국어의 발전 과정에 대해 계통, 문자체계부터 시작하여 고대국어부터 현대국어까지의 다양한 시대를 설명한 입문서이다."
1,9788973378869,78,34,"잃어버린 삶을 되찾도록 도와주는 위대한 역작 『몰입의 즐거움』은 전 세계 독자들에게 사랑받은 인문서의 바이블로, 일상을 변화시키는 행복 이론이다. 이 책은 현대에 맞게 한글 표기법을 수정하여 새로운 디자인으로 출간되었는데, 20주년을 맞아 스페셜 에디션과 전자책도 함께 출시되었다. 이 책은 몰입을 주목하고, 현재 하는 일에 몰입하는 것이 삶을 변화시키는 열쇠라고 강조한다. 국내에서도 20년 동안 사랑받아온 이 책은 인문 교양 베스트셀러로 자리 잡았고, 교수진들과 독자들로부터 높은 평가를 받았다."
2,9788937403224,54,20,1999년 제23회 오늘의 작가상 수상작 시간적 배경은 서기 2237년 공간적 배경은 가상체험 프로그램을 통해 떠난 19세기의 미국서부 플리머스라는 곳 여기에 각양각색의 인물들이 모여든다


#도서 검색
new_data에 [group 된 CID, lda 결과, 요약된 책 소개]

In [15]:
#검색어
query = "네 안에 잠든 거인을"

In [16]:
TTBKey = "ttbleejunho09091938001"

url = f"http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey={TTBKey}&Query={query}&QueryType=Title&MaxResults=10&start=1&SearchTarget=Book&output=JS&Version=20131101"
res = requests.get(url)
res.raise_for_status()

original_cid = res.json()['item'][0]['categoryId']
original_description = res.json()['item'][0]['description']

In [17]:
#1. CID
grouped_cid = depth2_dict[original_cid]

In [19]:
#2. LDA
#new_description 전처리
new_description = re.sub(r'[^\w\s가-힣]', ' ', original_description)  # 특수문자 제거
new_description = re.sub(r'[\n\d]', ' ', new_description)  # 줄바꿈, 숫자 제거

#Okt, 불용어사전 준비
Okt = konlpy.tag.Okt()
stopword_data_path = '/content/drive/MyDrive/졸업 프로젝트 - 도서추천/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/stopwords-ko.txt'
with open(stopword_data_path, 'r', encoding='utf-8') as f:
  stopwords = [line.strip() for line in f.readlines()]

#Okt, 불용어제거 실행
Okt_morphs = Okt.pos(new_description)
new_description = ' '.join([word for word, pos in Okt_morphs if pos == 'Noun' and word not in stopwords and len(word)>1])
new_description

'시대 최고 카운슬러 이자 행동 변화 전문가 토니 로빈 금연 다이어트 혁신 사회 성공 주체 목표 성공 신화 창조 근본 방법'

In [20]:
# 저장된 LDA 모델을 불러오기
lda_model = LdaModel.load('/content/drive/MyDrive/졸업 프로젝트 - 도서추천/도서분석 AI 용 데이터/최종 model/lda_model.model')

# 저장된 dictionary 파일을 불러오기
dictionary = Dictionary.load('/content/drive/MyDrive/졸업 프로젝트 - 도서추천/도서분석 AI 용 데이터/최종 model/dictionary.dict')

# 저장된 corpus 파일을 불러오기
corpus = corpora.MmCorpus('/content/drive/MyDrive/졸업 프로젝트 - 도서추천/도서분석 AI 용 데이터/최종 model/corpus.mm')

In [21]:
new_doc = new_description.split() #토큰화
new_doc2bow = dictionary.doc2bow(new_doc) #doc2bow
new_doc_lda = lda_model.get_document_topics(new_doc2bow) #lda 분석

for topic_id, prob in new_doc_lda:
    print(f"토픽 {topic_id}: 확률 {prob}")
#최대 확률 토픽
max_topic_id = max(new_doc_lda, key=lambda x: x[1])[0]

lda_topic = max_topic_id #저장
print(lda_topic)

토픽 0: 확률 0.03794841095805168
토픽 4: 확률 0.04990563169121742
토픽 10: 확률 0.5984473824501038
토픽 11: 확률 0.054417818784713745
토픽 18: 확률 0.19353258609771729
10


In [22]:
new_data = [grouped_cid, lda_topic, original_description]
print(new_data)

[81, 10, '이 시대 최고의 카운슬러이자 행동변화 전문가인 토니 로빈스. 금연, 다이어트에서부터 자기 혁신과 사회적 성공까지. 삶의 주체가 되어 분명한 목표를 세우고 성공 신화를 창조하는 근본적인 방법을 배울 수 있다.']


#책 소개 SBERT - FAISS 분석 : result_sbert
책 소개 데이터 Sentence BERT로 임베딩 후 FAISS분석

S-BERT로 임베딩

In [23]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 2. Sentence-BERT 모델 로드
#model = SentenceTransformer('all-MiniLM-L6-v2')  # 임베딩용 모델 선택
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

"""#임베딩 이미 만들어놔서 또 할 필요 없음
# 3. 임베딩 생성 (정수 결합 + 문장 임베딩)
def create_embedding(data):
    sentence_embedding = model.encode(data)  # 문장 임베딩

    return sentence_embedding

# 전체 데이터 임베딩 생성
embeddings = create_embedding(df['요약'])

print(embeddings.shape, df.shape)
np.save("/content/drive/MyDrive/도서분석 AI 용 12512데이터/s_bert_embeddings.npy", embeddings)

#3시간 19분 57초"""

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'#임베딩 이미 만들어놔서 또 할 필요 없음\n# 3. 임베딩 생성 (정수 결합 + 문장 임베딩)\ndef create_embedding(data):\n    sentence_embedding = model.encode(data)  # 문장 임베딩\n\n    return sentence_embedding\n\n# 전체 데이터 임베딩 생성\nembeddings = create_embedding(df[\'요약\'])\n\nprint(embeddings.shape, df.shape)\nnp.save("/content/drive/MyDrive/도서분석 AI 용 12512데이터/s_bert_embeddings.npy", embeddings)\n\n#3시간 19분 57초'

FAISS 사용

In [24]:
#embeddings 불러오기
embeddings = np.load("/content/drive/MyDrive/졸업 프로젝트 - 도서추천/도서분석 AI 용 데이터/s_bert_embeddings.npy")

In [37]:
# 4. FAISS 인덱스 생성
d = embeddings.shape[1]  # 벡터 차원
index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성
index.add(embeddings)  # 임베딩 데이터 추가

# 5. 새로운 데이터로 유사한 데 이터 검색
def find_similar_data(sentence, top_k=1):
  query_embedding = model.encode(sentence).reshape(1,-1) #-1은 자동으로 계산하라는 뜻.

  distances, similar_indices = index.search(query_embedding, top_k)  # 유사도 검색
  return distances, similar_indices

# 테스트
result_sbert_distances, result_sbert_indexes = find_similar_data(new_data[2], top_k=len(df))

In [ ]:
#index & 유사도
result_sbert = pd.DataFrame({
    'index': result_sbert_indexes[0],
    'faiss_distance': result_sbert_distances[0]
    })
result_sbert['faiss_similarity'] = np.max(result_sbert['faiss_distance']) - result_sbert['faiss_distance']
result_sbert

,index,faiss_distance,faiss_similarity
0,3233,131.795410,602.362549
1,17079,151.038742,583.119202
2,29354,152.108276,582.049683
3,36447,153.447311,580.710632
4,6571,154.673157,579.484802
...,...,...,...
42061,14426,665.740967,68.416992
42062,16277,665.740967,68.416992
42063,14546,700.103333,34.054626
42064,669,729.901001,4.256958


In [ ]:
df.iloc[result_sbert.head(5)['faiss_similarity'].values]

,ISBN,genre_code,lda_result,요약
602,9788982811852,54,16,소설은 전체적으로 사회 정치 경제 문화 전반에 걸친 부패할 대로 부패한 일본 사회...
583,9788986448276,69,0,자주 쓰이는 한자 어휘의 본래 의미를 되살려 한자로 대표되는 동양문화의 참뜻을 찾는...
582,9788972200239,78,34,저자는 미국의 경제활동과 그 밑바닥에 깔린 미국인들의 의식구조를 이해하기 위해 심리...
580,9788985466950,66,19,"현직 건축가가 유럽 여러 나라의 건축물을 탐방하면서 건축물의 의미, 예술성, 사연 ..."
579,9788976275691,0,60,이 책은 비주얼 C 를 이용하여 데이터베이스 애플리케이션을 개발해 본 경험이 전혀...


#LDA 유사도 분석

model의 전체 토픽 분포

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

"""# LDA 모델 학습 후 토픽 분포 저장
doc_topic_distributions = []
for doc in corpus:
    # 각 문서에 대한 토픽 분포 추출
    topic_distribution = lda_model.get_document_topics(doc, minimum_probability=0)
    # 확률 값만 추출해 리스트로 변환
    topic_distribution = [prob for _, prob in topic_distribution]
    doc_topic_distributions.append(topic_distribution)

np.save("/content/drive/MyDrive/도서분석 AI 용 데이터/최종 model/doc_topic_distributions.npy", doc_topic_distributions)"""

doc_topic_distributions = np.load("/content/drive/MyDrive/도서분석 AI 용 데이터/최종 model/doc_topic_distributions.npy")

새로들어온 데이터 lda 분석해 같은 형태로 전환

In [ ]:
new_doc = new_description.split() #토큰화
new_doc2bow = dictionary.doc2bow(new_doc) #doc2bow
new_doc_lda = lda_model.get_document_topics(new_doc2bow) #lda 분석

new_doc_topic_distribution = lda_model.get_document_topics(new_doc_lda, minimum_probability=0)
new_doc_topic_distribution = [prob for _, prob in new_doc_topic_distribution]

In [ ]:
similarities = cosine_similarity(
    [new_doc_topic_distribution],  # 새 문서의 토픽 분포
    doc_topic_distributions        # 기존 문서들의 토픽 분포
)

In [ ]:
#상위 100개 index & 유사도
result_lda = pd.DataFrame({
    'index': range(len(similarities[0])),
    'lda_similarity': similarities[0]
    })
result_lda

,index,lda_similarity
0,0,0.834401
1,1,0.103187
2,2,0.282258
3,3,0.281788
4,4,0.109676
...,...,...
42061,42061,0.101726
42062,42062,0.084665
42063,42063,0.109323
42064,42064,0.069239


In [ ]:
result_lda_sorted = result_lda.sort_values(by='lda_similarity', ascending=False)
df.iloc[result_lda_sorted.head(5)['index'].values]

,ISBN,genre_code,lda_result,요약
10743,9791156122814,65,20,30년대에 태어난 외할아버지와 90년대에 태어난 역사학도 손자가 손에 잡히는 '무명...
14731,9788976965288,65,16,한국의 60년 현대사 중 유권자 의지가 반영된 선거를 중심으로 한 강의가 있습니다....
31088,9788976130501,51,16,《타임TIME》가 담아낸 《타인의 시선》은 대한민국의 역동적인 현대사를 세계적인 관...
15245,9788952921031,61,20,왕이나 사건 중심의 역사 서술에서 벗어나 고려 시대 사람들의 삶의 모습을 생생하게 ...
24530,9788964372289,51,16,한국의 민주화운동과 관련한 책 『민중 만들기』는 1970년대와 80년대에 민중운동에...


#결과 내기

In [ ]:
lda_min = np.min(result_lda['lda_similarity'])
lda_max = np.max(result_lda['lda_similarity'])
result_lda['lda_norm'] = result_lda['lda_similarity'].apply(lambda x: (x - lda_min) / (lda_max - lda_min))

sbert_min = np.min(result_sbert['faiss_similarity'])
sbert_max = np.max(result_sbert['faiss_similarity'])
result_sbert['faiss_norm'] = result_sbert['faiss_similarity'].apply(lambda x: (x - sbert_min) / (sbert_max - sbert_min))

NameError: name 'result_lda' is not defined

In [ ]:
a = 0.75
b = 0.25

result_total = pd.merge(result_sbert, result_lda, on='index', how='inner')
result_total['total'] = (result_total['faiss_norm'] * a + result_total['lda_norm'] * b)*100
result_total = result_total.sort_values(by='total', ascending=False)
result_total.head(10)

NameError: name 'result_lda' is not defined

In [ ]:
df.iloc[result_total.head(10)['index'].values]

,ISBN,genre_code,lda_result,요약
35525,9791196661946,51,16,20년간 현대사회에서 인간의 마음을 연구한 소심한 심리학자가 제시한 '유연한 마음의...
35596,9791190257985,65,20,사회초년생부터 은퇴까지 경제 이야기 - 직장인들을 위한 필독서\n- 브런치에 기고 ...
12666,9788959060320,65,16,'이념의 시대'에서 '소비의 시대'로\n\n '이념의 시대'의 붕괴와 '소비의 시...
1684,9788984570009,0,16,99년 미국의 타임 과 뉴스위크 로부터 동시에 올해의 아시아 인물 로 선정되어...
3347,9788960517677,78,17,"""참동계""는 고수준의 수도공부 지침서로, 환골탈태와 깨침, 이론과 실전의 중요성을 ..."
28793,9791155426906,78,19,"『빅 히스토리』는 인류의 과거부터 미래까지를 조망하는 책으로, 과학, 역사, 종교,..."
15655,9788958283560,78,34,정치학자 강상중은 '고민하는 힘'이라는 책에서 삶을 살아가는데 드는 다양한 고민에 ...
13111,9788930081672,51,16,권태준 서울대 명예교수가 해방 이후로 현재까지의 한국현대사 전반을 돌아보면서 앞으로...
31380,9791160401868,51,16,한국의 바뀌고 있는 시대를 담은 책 《전환의 시대》는 대한민국의 ‘기본 골격’에 대...
18822,9788932471747,40,88,"앤서니 기든스의 『현대 사회학』은 사회학의 주요 주제와 이론을 다룬 책으로, 20년..."


In [ ]:
original_description

'이 시대 최고의 카운슬러이자 행동변화 전문가인 토니 로빈스. 금연, 다이어트에서부터 자기 혁신과 사회적 성공까지. 삶의 주체가 되어 분명한 목표를 세우고 성공 신화를 창조하는 근본적인 방법을 배울 수 있다.'